# Etudiantes
Lina Berroug
et
Jessica Karega

In [22]:
#IMPORTATION DE MODULES

import json
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [13]:
# Télécharger les stopwords en français si ce n'est pas déjà fait
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lnberroug/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Introduction
Ce notebook analyse les tweets d'un fichier JSON pour extraire des informations et les exploiter grâce à un dataFrame

Nous utiliserons les bibliothèques suivantes :
- `json` : pour manipuler les fichiers JSON
- `pandas` : pour analyser les données tabulaires
- `re` : pour extraire ou supprimer des expressions régulière d'un tweet
- `TextBlob` : pour l'analyse de sentiments
- `matplotlib` : pour visualiser sous forme de graphiques les données


### Diagramme des étapes de traitement
Schéma qui montre les étapes suivies pour le projet :

<img src="diagramme.png" alt="Description de l'image" style="width:40%; height:auto;">




In [14]:
#PARTIE : CREATION DU TWEET + SOUMISSION DU TWEET DANS LANDING ZONE

#Création d'une Classe Tweet à laquelle seront associées les attributs suivant : auteurs, hashtag, mentions, sentiments, (topics???)

class Tweet: #Plan/Modèle
    def __init__(self, contenu): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.contenu = contenu

    def Nettoyage_Tweet(self):
        """Fonction qui supprime les caractères spéciaux d'un tweet"""
        enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', self.contenu)
        return enlever_speciaux

class ZA:
    def __init__(self, id, auteur, texte): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.id = id
        self.auteur = auteur
        self.text = texte
        self.hashtags = self.Extraction_Hashtags()
        self.mentions = self.Extraction_Mentions()
        self.sentiment = self.Extraction_Sentiments()

    
    def Extraction_Sentiments(self): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(self.text)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
    def Extraction_Hashtags(self):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', self.text)
        return hashtags
    
    def Extraction_Mentions(self):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', self.text)

        return mentions


    def Id_Topic(tweet):

        def traitement_texte(text):
            text= text.lower() # mettre en minuscule
            text= re.sub(r'[^a-z\s]', '', text) 
            return text
        def get_stopwords():
            return stopwords.words('french')
        stopwords_list = get_stopwords()
        tweets= []
        with open ('versailles_tweets_100.json','r',encoding='utf-8') as f:
            donnee= json.load(f)
        
        tweets = [infos["text"] for infos in donnee]
        
        texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
        vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
        a = vecteur.fit_transform(texte_traite)

        # n_components=2 : Le modèle cherchera à identifier 2 sujets principaux dans les données textuelles.
        lda = LatentDirichletAllocation(n_components=2, random_state=42)
        lda.fit(a) # on utilise LDA sur la matrice X 

        topics = []
        # Récupération du vocabulaire extrait par TfidfVectorizer
        words = vecteur.get_feature_names_out()
        for i, topic_weights in enumerate(lda.components_):
            top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
            top_words = [words[i] for i in top_indices]  # Utilisation des indices pour récupérer les mots correspondants dans la liste `words`.

            topics.append(f": {', '.join(top_words)}")

        tweet_traite = traitement_texte(tweet)
        vecteur = TfidfVectorizer(stop_words= stopwords_list) # Utilise les stopwords en français intégrés dans sklearn
        vecteur_tweet= vecteur.transform([tweet_traite]) 
        probabilities = lda.transform(vecteur_tweet)
        topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

        print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic]}")


def Zone_Atterisssage (Fichier): # incomplet
        """Stockage du tweet dans le fichier json zone d'atterissage"""
        ZA_liste=[]
        
        with open (Fichier,'r',encoding='utf-8') as f:
            donnee= json.load(f)
            
        for infos_tweets in donnee:
            id = infos_tweets.get("id")
            auteur = infos_tweets.get("author_id")
            contenu = infos_tweets.get("text")

            tweet = Tweet(contenu)
            contenu_nettoye = tweet.Nettoyage_Tweet()
    

            tweet_analyse = ZA(id, auteur, contenu_nettoye)


            dict_tweet = {
                "id": tweet_analyse.id,
                "auteur": tweet_analyse.auteur,
                "tweet": contenu_nettoye,
                "texte": tweet_analyse.text,
                "hashtags" : tweet_analyse.hashtags,
                "mentions" : tweet_analyse.mentions,
                "sentiment" : tweet_analyse.sentiment
            }


            ZA_liste.append(dict_tweet)

        
        
        with open('ZA.json','w', encoding='utf-8') as f:
            json.dump(ZA_liste, f, indent=4, ensure_ascii=False)


# --> il faut charger ces données dans une date frame : df = pd.DataFrame(tweets)





In [15]:
#DATAFRAME
def CreerDataFrame(fichier_json):
    df  = pd.read_json(fichier_json)
    return df

#Analyses avancées 

def Top_K_element(df, colonne, k=3):
    """
    Retourne les K éléments les plus fréquents dans une colonne donnée.
    """
    top_k = df[colonne].explode().value_counts().head(k) #a méthode .explode() transforme chaque élément d'une liste dans une ligne distincte.
    element= top_k.index.tolist()  # Liste des hashtags
    counts = top_k.values.tolist()
    return element,counts

def publications_par_hashtag(df):
    """
    Retourne le nombre de publications par hashtag.
    """
    hashtags_explodes = df['hashtags'].explode()
    hashtag_counts = hashtags_explodes.value_counts()

    hashtags = hashtag_counts.index.tolist()  # Liste des hashtags
    counts = hashtag_counts.values.tolist()   # Liste des nombres d'occurrences
    return hashtags, counts

def publications_par_utilisateurs(df):
    """
    Retourne le nombre de publications par utilisateur.
    """
    utilisateur_counts = df['auteur'].value_counts()
    utilisateur = utilisateur_counts.index.tolist()  # Liste des utilisateurs
    counts = utilisateur_counts.values.tolist()   # Liste des nombres d'occurrences
    return utilisateur, counts

def publications_par_topic(df):
    """
    Retourne le nombre de publications par topics.
    """
    topics_counts = df['topics'].value_counts()
    topic = topics_counts.index.tolist()  # Liste des topics
    counts = topics_counts.values.tolist()   # Liste des nombres d'occurrences
    return topic , counts

def tweets_par_utilisateur(df,utilisateur):
    """
    Retourne le nombre de publications par topics.
    """
    filtre = df[df['auteur'] == utilisateur]
    tweets = filtre['contenu'].tolist() 
    utilisateurs = filtre ['auteur'].tolist() 
    return utilisateurs,tweets
    
def ensemble_de_tweets_mentionnant_utilisateur(df,utilisateur_mentionné):
    """
    Retourne le nombre de publications par topics.
    """
    filtre = df[df['mentions'] == utilisateur_mentionné]
    return filtre

def ensemble_de_utilisateurs_mentions_hashtag(df, hashtag):
    """
    Retourne le nombre de publications par topics.
    """
    filtre = df[df['hashtag'] == hashtag]
    return filtre
    
def ensemble_de_utilisateurs_mentionnés_utilisateur(df, utilisateur):
    """
    Retourne les mentions d'un utilisateur.
    """
    tweets_u = df[df['auteur'] == utilisateur]
    mentions = tweets_u['mentions'].explode()
    return list(mentions)




--> Pour améliorer ce code il faudrait utiliser nltk pour la tokenisation et la lemmenisation : ça va couper les tweet en petit mots et reduire les mots en leur forme conique ( je n'arrive pas à utilser nltk pour faire ça, erreur)

Ducoup la c pas précis

In [16]:
# Visualisation des résultats


def nbr_hashtag(df):
    hashtags, counts = publications_par_hashtag(df)
    plt.bar(hashtags,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel("Hashtags")
    plt.ylabel("Fréquence")
    plt.show()

def nbr_utilisateur(df):
    utilisateur, counts = publications_par_utilisateurs(df)
    plt.bar(utilisateur,counts, color = 'rouge')
    plt.title("Nombre de publication par utilisateur")
    plt.xlabel("utilisateurs")
    plt.ylabel("Fréquence")
    plt.show()

def nbr_topic(df):
    topic, counts = publications_par_topic(df)
    plt.bar(topic,counts, color = 'rouge')
    plt.title("Nombre de publication par topic")
    plt.xlabel("topics")
    plt.ylabel("Fréquence")
    plt.show()

def top_hashtgs(df):
    element,counts = Top_K_element(df, 'hashtags', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel(f"top {k} hashtags")
    plt.ylabel("Fréquence")
    plt.show()

def top_topic(df):
    element,counts = Top_K_element(df, 'topic', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel('topics')
    plt.ylabel("Fréquence")
    plt.show()

def top_auteur(df):
    element,counts = Top_K_element(df, 'auteur', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Top k utilisateur")
    plt.xlabel('Utilisateur')
    plt.ylabel("Fréquence")
    plt.show()

def top_hashtgs(df):
    element,counts = Top_K_element(df, 'mentions', k=3)
    plt.bar(element,counts, color = 'blue')
    plt.title("Nombre de publication par hashtag")
    plt.xlabel('mentions')
    plt.ylabel("Fréquence")
    plt.show()


    

In [17]:
t= "@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part , JO prochain on ira en demi final au moins. BRAVO à vous . #SupportriceMazo #domie #CIV"


def Nettoyage_Tweet(tweet):
    """Fonction qui supprime les caractères spéciaux d'un tweet"""
    enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', tweet)
    return enlever_speciaux

def Extraction_Sentiments(tweet): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(tweet)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
def Extraction_Hashtags(tweet):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', tweet)
        return hashtags
    
def Extraction_Mentions(tweet):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', tweet)
        return mentions


print(Nettoyage_Tweet(t))
print(Extraction_Sentiments(t))
print(Extraction_Hashtags(t))
print(Extraction_Mentions(t))




@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part  JO prochain on ira en demi final au moins BRAVO  vous  #SupportriceMazo #domie #CIV
neutre
['#SupportriceMazo', '#domie', '#CIV']
['@ericbailly24', '@maxigr04del']


In [63]:
import json

with open ('test.json','r',encoding='utf-8') as f:
    donne= json.load(f)

def Id_Topic(tweet):

    def traitement_texte(text):
        text= text.lower() # mettre en minuscule
        text= re.sub(r'[^a-z\s]', '', text) 
        return text
    def get_stopwords():
        return stopwords.words('english')
    stopwords_list = get_stopwords()
    tweets= []

        
    tweets = [infos["text"] for infos in donne]
        
    texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
    vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
    a = vecteur.fit_transform(texte_traite)

    # n_components=2 : Le modèle cherchera à identifier 2 sujets principaux dans les données textuelles.
    lda = LatentDirichletAllocation(n_components=2, random_state=42)
    lda.fit(a) # on utilise LDA sur la matrice X 

    topics = []
    # Récupération du vocabulaire extrait par TfidfVectorizer
    words = vecteur.get_feature_names_out()
    for i, topic_weights in enumerate(lda.components_):
        top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
        top_words = [words[i] for i in top_indices]  # Utilisation des indices pour récupérer les mots correspondants dans la liste `words`.
        topics.append(f": {', '.join(top_words)}")

    tweet_traite = traitement_texte(tweet)
    vecteur_tweet = vecteur.transform([tweet_traite])
    probabilities = lda.transform(vecteur_tweet)
    topic_idx = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

    print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic_idx ]}")

tweet = "A balanced diet can prevent many diseases."

Id_Topic(tweet)


Le tweet :
A balanced diet can prevent many diseases.
Appartient au Topic : year, world


In [86]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
import nltk

# Télécharger les stopwords si nécessaire
nltk.download('stopwords')

# Définir les thèmes principaux avec leurs mots-clés caractéristiques
themes = {
    "Technologie": ["smartphone", "ai", "artificial", "intelligence", "tesla", "5g", "autonomous", "future", "vehicle", "robot", "machine", "algorithm", "data", "programming"],
    "Sports": ["gold", "olympics", "champions", "league", "serena", "athletes", "football", "team", "match", "qualifiers", "score", "tournament", "basketball", "goal"],
    "Santé": ["mental", "health", "vaccination", "fasting", "diet", "exercise", "life", "disease", "awareness", "nutrition", "wellness", "hospital", "doctor", "medicine"],
    "Divertissement": ["adele", "album", "marvel", "movie", "series", "netflix", "oscars", "swift", "tour", "boxoffice", "music", "concert", "theater", "festival"],
    "Politique": ["election", "government", "policy", "minister", "law", "debate", "vote", "campaign", "president", "parliament", "senate", "democracy"],
    "Économie": ["market", "stock", "economy", "finance", "investment", "growth", "business", "trade", "money", "tax", "employment", "entrepreneur"],
    "Éducation": ["school", "university", "student", "teacher", "class", "learning", "exam", "homework", "course", "degree", "education", "knowledge", "scholarship"],
    "Climat": ["climate", "global", "warming", "environment", "sustainability", "pollution", "green", "renewable", "recycle", "weather", "carbon", "planet", "biodiversity"],
    "Voyages": ["travel", "trip", "vacation", "flight", "hotel", "destination", "adventure", "explore", "tourism", "cruise", "passport", "beach", "holiday"],
    "Cuisine": ["recipe", "dish", "meal", "cooking", "baking", "chef", "restaurant", "kitchen", "ingredients", "food", "menu", "flavor", "dinner", "lunch"]
}


# Charger les données
with open('test.json', 'r', encoding='utf-8') as f:
    donne = json.load(f)

def Id_Topic(tweet):
    # Fonction pour nettoyer le texte
    def traitement_texte(text):
        text = text.lower()  # Convertir en minuscule
        text = re.sub(r'[^a-z\s]', '', text)  # Supprimer les caractères spéciaux
        return text

    def get_stopwords():
        return stopwords.words('french')

    stopwords_list = get_stopwords()
    tweets = [infos["text"] for infos in donne]

    # Nettoyer les textes des tweets
    texte_traite = [traitement_texte(tweet) for tweet in tweets]

    # Initialiser et entraîner le vecteur TF-IDF
    vecteur = TfidfVectorizer(stop_words=stopwords_list)
    a = vecteur.fit_transform(texte_traite)

    # Entraîner le modèle LDA
    lda = LatentDirichletAllocation(n_components=20, random_state=42)
    lda.fit(a)

    # Récupérer les topics
    topics = []
    words = vecteur.get_feature_names_out()
    for i, topic_weights in enumerate(lda.components_):
        top_indices = topic_weights.argsort()[:-6:-1]  # Prendre les 2 mots les plus importants
        top_words = [words[i] for i in top_indices]
        topics.append(top_words)

    # Prédire le topic pour le tweet donné
    tweet_traite = traitement_texte(tweet)
    vecteur_tweet = vecteur.transform([tweet_traite])
    probabilities = lda.transform(vecteur_tweet)
    topic_idx = probabilities.argmax()  # Trouver l'indice du topic avec la probabilité la plus élevée
    topic_words = topics[topic_idx]  # Mots principaux du topic

    # Identifier le thème principal
    theme_scores = {}
    for theme, keywords in themes.items():
        common_words = set(topic_words).intersection(set(keywords))
        theme_scores[theme] = len(common_words)

    # Ajouter un thème "Inconnu" si aucun mot ne correspond
    if max(theme_scores.values()) == 0:
        closest_theme = "Inconnu"
    else:
        closest_theme = max(theme_scores, key=theme_scores.get)  # Trouver le thème avec le plus grand score

    # Résultats
    print(f"Le tweet :\n{tweet}")
    print(f"Mots principaux du Topic {topic_idx} : {', '.join(topic_words)}")
    print(f"Thème principal estimé : {closest_theme} (Basé sur les mots clés des topics)")

# Exemple de tweet
tweet = "Intermittent fasting shows great health benefits."
Id_Topic(tweet)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Le tweet :
Intermittent fasting shows great health benefits.
Mots principaux du Topic 3 : great, fasting, benefits, shows, intermittent
Thème principal estimé : Santé (Basé sur les mots clés des topics)


In [85]:
import re
from textblob import TextBlob

# Liste de mots-clés pour les topics
TOPICS = {
    "technologie": ["AI", "machine learning", "cloud", "blockchain"],
    "environnement": ["écologie", "biodiversité", "climat", "pollution"],
    "santé": ["médecine", "vaccin", "hôpital", "maladie"],
    "politique": ["élection", "gouvernement", "loi", "politique"]
}

def id_topic(tweet_text):

    """Identifie les topics d'une publication en fonction de mots-clés."""
    topics_found = []
    
    # Nettoyer le texte du tweet
    clean_text = re.sub(r"[^\w\s]", "", tweet_text.lower())
    
    # Parcourir les topics et leurs mots-clés
    for topic, keywords in TOPICS.items():
        for keyword in keywords:
            if keyword.lower() in clean_text:
                topics_found.append(topic)
                break  # On évite de répéter le même topic
    
    return topics_found if topics_found else ["non catégorisé"]

# Exemple d'utilisation
tweet = "I'm at Gardens of Versailles in Versailles"
result = id_topic(tweet)
print("Topics identifiés :", result)


Topics identifiés : ['technologie']
